In [23]:
import cv2
import os
import time
import json

from ultralytics import YOLO


def dhash(image, hash_size=8):
    if image is None or image.size == 0:
        return None

    resized = cv2.resize(image, (hash_size + 1, hash_size))
    diff = resized[:, 1:] > resized[:, :-1]
    return sum([2**i for (i, v) in enumerate(diff.flatten()) if v])


# Load an official or custom model
model = YOLO("yolov8m.pt")  # Load an official Detect model

# Perform tracking with the model
results = model.track(
    "dataset/freerolls/Recording 2024-07-09 152608.mp4",
    tracker="bytetrack.yaml",
    stream=True,
    verbose=False,
)  # with ByteTrack

# Directory where you want to save the frames
save_dir = "./dataset/freerolls/frames/"
os.makedirs(save_dir, exist_ok=True)

# JSONL file
jsonl_file = os.path.join("./dataset/freerolls/approvals", "approval.jsonl")

# Empty JSONL file
with open(jsonl_file, "w") as f:
    f.write("")

lines = []

"""for i in indices:
        x, y, w, h = boxes[i]

        detected_region = image[y:y+h, x:x+w]
        hash_value = dhash(detected_region,16)

        # Add information to JSON array
        json_objects.append({
            "h": str(hash_value),
            "cs": classes[class_ids[i]],
            "cf": round(confidences[i],2),
            "cd": {"x": x, "y": y, "w": w, "h": h}
        })

    imageHash = dhash(image,1)

    # Final Payload
    data = {
        "status": True,
        "id": id_portion,
        "ts": ts,
        "ihash":imageHash,
        "obj": len(json_objects),
        "detect": json_objects,
    }
"""

frame_count = 0
for result in results:
    image = result.orig_img.copy()

    # Get current Unix time in milliseconds
    unix_time_ms = int(time.time() * 1000)

    id = f"{unix_time_ms}_{frame_count:04d}"

    # Save frame as JPEG file
    save_path = os.path.join(save_dir, f"{id}.jpg")
    cv2.imwrite(save_path, image)
    frame_count += 1

    boxes = result.boxes

    json_objects = []
    for box in boxes:
        # Get bounding box coordinates as int32
        x1, y1, x2, y2 = list(map(int, box.xyxy[0].cpu().numpy()))
        
        scores = float(box.conf)
        class_id = int(box.cls)
        class_name = model.names[class_id]

        if class_name == "person":
            # Crop detected region
            detected_region = image[y1:y2, x1:x2]
            # cropped_path = os.path.join(save_dir, f"{unix_time_ms}_{frame_count:04d}_cropped_{box.id}.jpg")
            # cv2.imwrite(cropped_path, detected_region)

            hash_value = dhash(detected_region, 16)
            
            # Find height and width of the detected region
            h, w, _ = detected_region.shape

            json_objects.append(
                {
                    "h": str(hash_value),
                    "cs": class_id,
                    "cf": round(scores, 2),
                    "cd": {"x": x1, "y": y1, "w": w, "h": h},
                }
            )

    imageHash = dhash(image, 1)

    line = {
        "status": True,
        "id": id,
        "ts": unix_time_ms,
        "ihash": str(imageHash),
        "obj": len(boxes),
        "detect": json_objects,
    }
    
    print(f'id => {id}')

    json_string = json.dumps(line)
    lines.append(json_string)

with open(jsonl_file, "w") as f:
    for line in lines:
        json_string = json.dumps(line)
        f.write(json_string + '\n')

id => 1720587532055_0000
id => 1720587532103_0001
id => 1720587532151_0002
id => 1720587532199_0003
id => 1720587532244_0004
id => 1720587532290_0005
id => 1720587532336_0006
id => 1720587532380_0007
id => 1720587532425_0008
id => 1720587532477_0009
id => 1720587532525_0010
id => 1720587532573_0011
id => 1720587532627_0012
id => 1720587532673_0013
id => 1720587532720_0014
id => 1720587532768_0015
id => 1720587532814_0016
id => 1720587532859_0017
id => 1720587532903_0018
id => 1720587532947_0019
id => 1720587532991_0020
id => 1720587533035_0021
id => 1720587533079_0022
id => 1720587533137_0023
id => 1720587533206_0024
id => 1720587533252_0025
id => 1720587533298_0026
id => 1720587533342_0027
id => 1720587533387_0028
id => 1720587533430_0029
id => 1720587533476_0030
id => 1720587533523_0031
id => 1720587533571_0032
id => 1720587533616_0033
id => 1720587533663_0034
id => 1720587533709_0035
id => 1720587533754_0036
id => 1720587533797_0037
id => 1720587533843_0038
id => 1720587533887_0039


In [1]:
import gc
import torch

del model
del results
gc.collect()
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [5]:
import cv2
import os
import time

# Path to the video file
video_path = './dataset/freerolls/Recording 2024-07-09 152608.mp4'

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error opening video file")

# Directory where you want to save the frames
save_dir = './dataset/freerolls/frames/'
os.makedirs(save_dir, exist_ok=True)

frame_count = 0
while True:
    # Read a frame
    ret, frame = cap.read()

    # If read was successful
    if ret:
        # Get current Unix time in milliseconds
        unix_time_ms = int(time.time() * 1000)

        # Save frame as JPEG file
        save_path = os.path.join(save_dir, f"{unix_time_ms}_{frame_count:04d}.jpg")
        cv2.imwrite(save_path, frame)
        frame_count += 1
    else:
        break

# Release the VideoCapture object
cap.release()
print(f"Extracted {frame_count} frames and saved to {save_dir}")

Extracted 7668 frames and saved to ./dataset/freerolls/frames/
